In [ ]:
import openai
from openai import OpenAI
import langchain
import os
# Step 1: Load environment variables
%reload_ext dotenv
%dotenv
# Step 3: Access your OpenAI API key from environment variables
for key, value in os.environ.items():
    if key == "OPENAI_API_KEY":
        print(f"{key}: value")

OPENAI_API_KEY: value


In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(model='gpt-4o-mini', temperature=0,max_completion_tokens=50)
# response = chat.invoke("I have recently adopted a dog. Could you suggest some dog names?")
# print(response.content)

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.messages import SystemMessage
from langchain_core.prompts import (ChatPromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    MessagesPlaceholder,
                                    PromptTemplate)

from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import (ConversationBufferMemory,
                              ConversationBufferWindowMemory,
                              ConversationSummaryMemory,
                              CombinedMemory)

from langchain.globals import set_verbose
set_verbose(True)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

# Define the system message
sys_message = SystemMessage(content='''The following is a friendly conversation between a human and an AI. 
If the AI does not know the answer to a question, it truthfully says it does not know.''')

# Define message placeholders for both buffer and summary
buffer_placeholder = MessagesPlaceholder(variable_name='message_buffer_log')
summary_placeholder = MessagesPlaceholder(variable_name='message_summary_log') 
hum_message_template = HumanMessagePromptTemplate.from_template(template='{question}')

# Create the chat prompt template
chat_template = ChatPromptTemplate.from_messages([
    sys_message,
    buffer_placeholder,
    summary_placeholder,
    hum_message_template
])

# Initialize memories (note return_messages=True for both)
chat_buffer_memory = ConversationBufferMemory(
    memory_key='message_buffer_log',
    input_key='question',
    return_messages=True  # Changed to True for MessagesPlaceholder
)

chat_summary_memory = ConversationSummaryMemory(
    llm=chat,
    memory_key='message_summary_log',
    input_key='question',
    return_messages=True  # Changed to True for MessagesPlaceholder
)

chat_combined_memory = CombinedMemory(memories=[chat_buffer_memory, chat_summary_memory])

# Create the chain
chain = LLMChain(
    llm=chat,
    prompt=chat_template,
    memory=chat_combined_memory,
    verbose=True  # Optional for debugging
)

# Example usage
response = chain.invoke({'question': "Can you give me an interesting fact I probably didn't know about?"})
print(response['text'])

response = chain.invoke({'question': "Can you give me another interesting fact I probably didn't know about?"})
print(response['text'])

response = chain.invoke({'question': "Can you talk about Rodri?"})
print(response['text'])

# Check memory states
print("\nBuffer Memory:")
print(chat_buffer_memory.load_memory_variables({}))

print("\nSummary Memory:")
print(chat_summary_memory.load_memory_variables({}))

print("\nCombined Memory:")
combined_memory = chat_combined_memory.load_memory_variables({})
print(combined_memory)
print("\nSummary from Combined Memory:")
print(combined_memory['message_summary_log'])